In [1]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from mSDA import msda
from utils.adaptacion import *

#otros
import numpy as np
import pandas as pd

tipo = pruebas[1]
dataset_name = datasets[0]
dims = dimensions[dataset_name]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Pruebas con el dataset Amazon

In [2]:
print tipo
print dataset_name
print dims
print data_path

msda
amazon
5000
data


In [3]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
unlabeled = dataset_object.unlabeled
domains = dataset_object.domains

Dataset already splitted


In [4]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X)

print "Todos los datos disponibles obtenidos"

Todos los datos disponibles obtenidos


```python

saved_paths = [ruta_1,...,ruta_n]
```

In [ ]:
msda_paths = os.path.join(models_path, tipo, "%s_paths.pkl" % dataset_name)

if os.path.exists(msda_paths):
    print "Cargando rutas de modelos adaptados."
    saved_paths = joblib.load(msda_paths)
else:
    print "Creando modelos de adaptacion..."

    noises = [0.3, 0.5, 0.8]
    layers_sizes = [1, 3, 5]

    parametros = {
        'noises': noises,
        'layers': layers_sizes
    }


    saved_paths = msda_pseudo_grid_search(X, parametros, models_path, tipo, dataset_name)

    print "Modelos creados.\n"
    
    save_adapted_model(saved_paths, msda_paths)
    print "Guardando rutas en %s" % msda_paths

Creando modelos de adaptacion...
pr: 0.300 - l: 1


```python
domain_msda_paths = [ruta_1,...,ruta_n]
```

In [ ]:
best_msda_paths = os.path.join(models_path, tipo, "best_%s_paths.pkl" % dataset_name )
if os.path.exists(best_msda_paths):
    #se cargan las rutas de los modelos guardados
    domain_msda_paths = joblib.load(best_msda_paths)   
else:
    domain_msda_paths = []
    #se obtienen los mejores modelos 
    for domain in domains:
        print "Obteniendo mejor clasificador para %s" % domain
        i = 1

        best_score = 0
        best_model = None
        for msda_model_path in saved_paths:
            #print "Modelo %d de %d" % (i, len(saved_paths))


            msda_model = joblib.load(msda_model_path)
            mapping = msda_model['mapping']
            pr = msda_model['pr']
            n_layers = msda_model['l']

            print "l=%d, pr=%.3f " % (msda_model['l'], msda_model['pr']),

            # se obtienen los datos del dominio
            X_tr = np.asarray(labeled[domain]['X_tr'][:, :dims].todense())
            y_tr = np.asarray(labeled[domain]['y_tr'].todense()).argmax(axis=1)

            # se adaptan los datos
            X_deep = msda.mSDA(X_tr, pr, n_layers, mapping)[1][:][:][-1]

            # se obtiene el mejor score con GridSearch
            new_clf = get_best_score(X_deep, y_tr, clasifier='SVC')
            new_score = new_clf.best_score_
            print "score: %.4f" % new_score
            # se guarda si es el mejor para el modelo
            if new_score > best_score:
                best_score = new_score
                best_model = msda_model

            i = i+1

        #se guarda el mejor modelo para este dominio
        print "Mejor modelo: l=%d, pr=%.3f" % (best_model['l'], best_model['pr'])
        best_msda_path = os.path.join(models_path, tipo, "%s_%s.pkl" % (dataset_name, domain))
        print "Guardando en %s" % best_msda_path
        joblib.dump(best_model, best_msda_path)
        
        domain_msda_paths.append(best_msda_path)
    
    print "Guardando rutas en %s" % best_msda_paths
    joblib.dump(domain_msda_paths, best_msda_paths)

In [ ]:
domain_msda_paths

In [ ]:
#diccionario para mantener los dominios adaptados
adapted = {}

i = 0
for domain in domains:
    best_msda_path = os.path.join(models_path, tipo, "%s_%s.pkl" % (dataset_name, domain))
    
    if os.path.exists(best_msda_path):
        print "Cargando mejor modelo para %s" % domain
        msda_model = joblib.load(best_msda_path)
        mapping = msda_model['mapping']
        
        print "Adaptando dominio"
        X_tr = np.asarray(labeled[domain]['X_tr'].todense())
        X_ts = np.asarray(labeled[domain]['X_ts'].todense())
        
        
        tr_reps = msda.mSDA(X_tr, pr, n_layers, mapping)[1][:][:][-1]
        ts_reps = msda.mSDA(X_ts, pr, n_layers, mapping)[1][:][:][-1]

        adapted[domain] = {
            'X_tr': tr_reps,
            'X_ts': ts_reps
        }
        
        i = i+1
    else:
        print "Generar mejor modelo para %s" % domain
        
print "%d/%d dominios adaptados" % (i, len(domains))

In [ ]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de 12" % (i+1)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'][:, :dims].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'][:, :dims].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            #model_name = "%s_%s_%s.pkl" % (tipo,src, tgt)
            model_name = "indomain_%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, model_name)
            
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            #svc = get_best_score(X_tr, y_tr)
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            
            #transfer error
            #entrenado en dominio src y probado en dominio tgt adaptados
            X_tr_a = adapted[src]['X_tr']
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts_a = adapted[tgt]['X_ts']
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            print "Entrenando clasificador adaptado."
            svc_a = get_best_score(X_tr_a, y_tr)
            
            t_error = 1-svc_a.score(X_ts_a, y_ts)
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['mSDA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            
            i+=1
    
print "Pruebas completadas"

In [ ]:
df

In [ ]:
new_scores_path = os.path.join(scores_path, tipo, dataset_name+".csv")
print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."